### Connect to database

In [96]:
import mariadb # librairie pour se connecter à mariadb
import sys
import var # import variable configured into var.py


# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user=var.user,
        password=var.password,
        host=var.host,
        port=3306,
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()


### Retrieve databases on the server
❗❗<code>SHOW DATABASES</code> query requires specific right on mariadb server
If our user do not get those rights, please run the command with another user
Some databases are not supposed to be transfered into Elastic, for exemple : <code>{'information_schema','mysql','sys','performance_schema'}</code>


In [97]:
cur.execute("SHOW DATABASES;") #* require specific rights on mariadb server

result = cur.fetchall() # retrieve all the results

cur.close()
# Create a list of databases name on the server
dbs_list= []
for row in result:
    dbs_list.append(row[0]) 

#! TODO : complete unwanted_db with any database that should not be transfered into Elastic
unwanted_db = {'information_schema','mysql','sys','performance_schema'}

#* dbs_list contain all databases that will be taken into account for batch
dbs_list = [db for db in dbs_list if db not in unwanted_db]

In [98]:
display(dbs_list)

['meteo1', 'mouleconnected']

### Get tables inside Database
We create a dictionnary <code>dict_db</code> with database name as key and value being a list  of tables inside the database

In [99]:
# for each db we want to transfer, we store tables names in the dictionnary
dict_db = {}
for db in dbs_list:
    cur = conn.cursor()
    query = f"SHOW TABLES FROM {db}"
    cur.execute(query)
    result = cur.fetchall()
    result = [value[0] for value in result] # keep only the name of the table
    dict_db[db] = result
    cur.close()


In [100]:
display(dict_db)

{'meteo1': ['DATA',
  'LOGBOOK',
  'ane0',
  'bar0',
  'cpr0',
  'gps0',
  'gsm0',
  'ntp0',
  'rad0',
  'the0'],
 'mouleconnected': ['Temperature',
  'bac',
  'capteur',
  'data',
  'data2',
  'lumiere',
  'moule',
  'users']}

Convert dict of list into dict of dict with list for values extracted

In [101]:
dict_db_tables = {}
for key, values in dict_db.items():
    dict_db_tables[key] = {value: [] for value in values}
display(dict_db_tables)

{'meteo1': {'DATA': [],
  'LOGBOOK': [],
  'ane0': [],
  'bar0': [],
  'cpr0': [],
  'gps0': [],
  'gsm0': [],
  'ntp0': [],
  'rad0': [],
  'the0': []},
 'mouleconnected': {'Temperature': [],
  'bac': [],
  'capteur': [],
  'data': [],
  'data2': [],
  'lumiere': [],
  'moule': [],
  'users': []}}

### Retrieve columns in tables

In [106]:
for db in dict_db.keys():
    #print(db)
    for table in dict_db[db]:
        cur = conn.cursor()
        query = f"SHOW COLUMNS FROM {db}.{table}"
        cur.execute(query)
        result = cur.fetchall()
        cols = [value[0] for value in result]
        i=0
        for i in range(len(cols)):
            dict_db_tables[db][table][cols[i]] = {}

display(dict_db_tables)
        

TypeError: list indices must be integers or slices, not str

### Extract data from table
#### mouleconnected


In [103]:
for db in dict_db.keys():
    #print(db)
    for table in dict_db[db]:
        cur = conn.cursor()
        query = f"SELECT * FROM {db}.{table} LIMIT 10"
        cur.execute(query)
        result = cur.fetchall()
        print(table,result)
        dict_db_tables[db][table] = result
display(dict_db_tables)

{'meteo1': {'DATA': [(20091003092549, 0.0),
   (20091003093001, 0.0),
   (20091003093527, 0.0),
   (20091003094002, 0.0),
   (20091003095001, 0.0),
   (20091003100002, 0.0),
   (20091003101001, 0.0),
   (20091003102001, 0.0),
   (20091003103001, 0.0),
   (20091003104002, 0.0)],
  'LOGBOOK': [],
  'ane0': [(20080413114420, -99999.0, -99999.0),
   (20080413114430, 7.1, 114.0),
   (20080413114440, 7.0, 123.0),
   (20080413114450, 8.3, 123.0),
   (20080413114500, 5.2, 110.0),
   (20080413114510, 10.9, 112.0),
   (20080413114520, 11.0, 134.0),
   (20080413114530, 12.9, 124.0),
   (20080413114540, 9.7, 116.0),
   (20080413114550, 10.0, 110.0)],
  'bar0': [(20080413114420, -99999.0),
   (20080413114930, 1018.0),
   (20080413115431, 1018.0),
   (20080413115931, 1018.0),
   (20080413120431, 1018.0),
   (20080413120932, 1018.0),
   (20080413121748, -99999.0),
   (20080413122258, 1018.0),
   (20080413122759, 1018.0),
   (20080413123259, 1018.0)],
  'cpr0': [(20080413114418, -99999.0, -99999.0, -9

### JOIN tables